This script computes emissions within city boundaries using standard Soundcast network outputs and established emissions rates for base and forecast years. Additional functions are provided from the imported "emissions" and "functions" methods.

In [6]:
import os
import pandas as pd
import geopandas as gpd
import os.path

from functions import read_from_sde, load_network_summary, intersect_geog
from emissions import *

In [7]:
# Set root of model run to analyze AND the model year
run_dir = r'\\modelstation1\c$\workspace\sc_2018_test_jo'
model_year = '2018'    # Make sure to update this since rates used are based on this value
# run_dir = r'\\modelstation1\c$\workspace\sc_rtp_2030_final\soundcast'
# model_year = '2030'    # Make sure to update this since rates used are based on this value
# run_dir = r'\\modelstation1\c$\workspace\sc_2040_rtp_final\soundcast'
# model_year = '2040'
# run_dir = r'\\modelstation1\c$\workspace\sc_rtp_2050_constrained_final\soundcast'
# model_year = '2050'

# Set output directory; results will be stored in a folder by model year 
output_dir = r'C:\Workspace\aq_tool\output_burien\\' + model_year

In [8]:
# Create outputs directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Change working directory to run_dir
os.chdir(run_dir)

# Load the network
crs = 'EPSG:2285'
gdf_network = gpd.read_file(os.path.join(run_dir,r'inputs\scenario\networks\shapefiles\AM\AM_edges.shp'))
gdf_network.crs = crs

In [9]:
# Load  tract geographies from ElmerGeo
connection_string = 'mssql+pyodbc://AWS-PROD-SQL\Sockeye/ElmerGeo?driver=SQL Server?Trusted_Connection=yes'

version = "'DBO.Default'"
gdf_shp = read_from_sde(connection_string, 'regional_geographies_preferred_alternative', version, crs=crs, is_table=False)

In [10]:
# Load parcels as a geodataframe
parcel_df = pd.read_csv(os.path.join(run_dir,r'inputs/scenario/landuse/parcels_urbansim.txt'), delim_whitespace=True,
                            usecols=['PARCELID','XCOORD_P','YCOORD_P'])

parcel_gdf = gpd.GeoDataFrame(parcel_df,
        geometry=gpd.points_from_xy(parcel_df['XCOORD_P'], parcel_df['YCOORD_P']), crs=crs)

In [11]:
# Perform intersect to get the network within each city in a list
df_network = load_network_summary(os.path.join(run_dir, r'outputs\network\network_results.csv'))


In [12]:

def evaluate_emissions(_gdf_shp, model_year):
    """Compute emissions for a jurisdiction. 
    
        _gdf_shp: polygon geodataframe of a jurisidiction (should be complete coverage without holes over bodies of water, etc.)
        model_year: must be passed for clarity

    """

    # Intersect jurisdiction polygon with network shapefile and network CSV file
    _gdf_shp = intersect_geog(_gdf_shp, gdf_network, df_network)

    # Select links from network summary dataframe that are within the gdf_shp
    # The dataframe contains link-level model outputs
    _df = df_network[df_network['ij'].isin(_gdf_shp['id'])]

    # Replace length with length from _gdf_shp to ensure roads stop at city boundaries
    # Drop "length field from _gdf_shp, which is length in miles; 
    # use new_length field, which is calculated length of intersected links in feet
    _df.drop('length', axis=1, inplace=True)
    _df = _df.merge(_gdf_shp, how='left', left_on='ij', right_on='id')
    # _df.drop('length', axis=1, inplace=True)
    _df.rename(columns={'new_length': 'length',
                    'length': 'original_length'}, inplace=True)

    # Calculate interzonal emissions using same approach as for regional/county emissions
    os.chdir(run_dir)
    conn = create_engine('sqlite:///inputs/db/soundcast_inputs.db')

    # Load running emission rates by vehicle type, for the model year
    df_running_rates = pd.read_sql('SELECT * FROM running_emission_rates_by_veh_type WHERE year=='+model_year, con=conn)
    df_running_rates.rename(columns={'ratePerDistance': 'grams_per_mile'}, inplace=True)
    df_running_rates['year'] = df_running_rates['year'].astype('str')

    # Select the month to use for each pollutant; some rates are used for winter or summer depending
    # on when the impacts are at a maximum due to temperature.
    df_summer = df_running_rates[df_running_rates['pollutantID'].isin(summer_list)]
    df_summer = df_summer[df_summer['monthID'] == 7]
    df_winter = df_running_rates[~df_running_rates['pollutantID'].isin(summer_list)]
    df_winter = df_winter[df_winter['monthID'] == 1]
    df_running_rates = df_winter.append(df_summer)

    df_interzonal_vmt = calculate_interzonal_vmt(_df)
    df_interzonal = calculate_interzonal_emissions(df_interzonal_vmt, df_running_rates)

    # Get list of zones to include for intrazonal trips
    # Include intrazonal trips for any TAZ centroids within city boundary

    # Load TAZ centroids
    connection_string = 'mssql+pyodbc://AWS-PROD-SQL\Sockeye/ElmerGeo?driver=SQL Server?Trusted_Connection=yes'

    version = "'DBO.Default'"
    taz_gdf = read_from_sde(connection_string, 'taz2010_no_water', version, crs=crs, is_table=False)

    _taz_gdf = gpd.GeoDataFrame(taz_gdf.centroid)
    _taz_gdf.geometry = _taz_gdf[0]
    _taz_gdf['taz'] = taz_gdf['taz'].astype('int')

    city_gdf = gdf_shp[gdf_shp['juris'] == city]

    intersect_gdf = gpd.overlay(_taz_gdf, 
            city_gdf, 
            how="intersection")

    # Load intrazonal trips for zones in the area
    df_iz = pd.read_csv(os.path.join(run_dir,r'outputs\network\iz_vol.csv'))

    # Filter for zone centroids within the jurisdiction
    _df_iz = df_iz[df_iz['taz'].isin(intersect_gdf.taz)]

    # If no zones centroids in a city, pass an empty df with 0 values for VMT;
    # Otherwise calculate intrazonal VMT for the associated TAZs only
    if len(_df_iz) > 0:
        df_intrazonal_vmt = calculate_intrazonal_vmt(_df_iz, conn)
        
    else:
        # Load the regional results and fill VMT with 0
        df_intrazonal_vmt = pd.read_csv(os.path.join(run_dir, r'outputs\emissions\intrazonal_vmt_grouped.csv'))
        df_intrazonal_vmt['VMT'] = 0
    df_intrazonal = calculate_intrazonal_emissions(df_intrazonal_vmt, df_running_rates)

    # Intersect parcels with the city gdf to get number of household to adjust vehicle starts
    parcel_intersect_gdf = gpd.overlay(parcel_gdf,  
                city_gdf, 
                how="intersection")
    start_emissions_df = calculate_start_emissions(conn, parcel_intersect_gdf, model_year)

    df_inter_group = df_interzonal.groupby(['pollutantID','veh_type']).sum()[['tons_tot','vmt']].reset_index()
    df_inter_group.rename(columns={'tons_tot': 'interzonal_tons', 'vmt': 'interzonal_vmt'}, inplace=True)
    df_intra_group = df_intrazonal.groupby(['pollutantID','veh_type']).sum()[['tons_tot','vmt']].reset_index()
    df_intra_group.rename(columns={'tons_tot': 'intrazonal_tons', 'vmt': 'intrazonal_vmt'}, inplace=True)
    df_start_group = start_emissions_df.groupby(['pollutantID','veh_type']).sum()[['start_tons']].reset_index()

    summary_df = pd.merge(df_inter_group, df_intra_group, how='left').fillna(0)
    summary_df = pd.merge(summary_df, df_start_group, how='left')
    summary_df = finalize_emissions(summary_df, col_suffix="")
    summary_df.loc[~summary_df['pollutantID'].isin(['PM','PM10','PM25']),'pollutantID'] = summary_df[~summary_df['pollutantID'].isin(['PM','PM10','PM25'])]['pollutantID'].astype('int')
    summary_df['pollutant_name'] = summary_df['pollutantID'].astype('int', errors='ignore').astype('str').map(pollutant_map)
    summary_df['total_daily_tons'] = summary_df['start_tons']+summary_df['interzonal_tons']+summary_df['intrazonal_tons']
    summary_df = summary_df[['pollutantID','pollutant_name','veh_type','intrazonal_vmt','interzonal_vmt','start_tons','intrazonal_tons','interzonal_tons','total_daily_tons']]

    return summary_df


In [13]:
# df_interzonal_vmt[['sov_vmt','hov2_vmt','hov3_vmt','tnc_vmt','bus_vmt','medium_truck_vmt','heavy_truck_vmt']].sum().sum()

In [14]:
# Select all cities and towns in King county
# king_city_list = gdf_shp[(gdf_shp['cnty_name'] == 'King') & (gdf_shp['rg_propose_pa'].isin(['CitiesTowns', 'Core',
#                 'Metro','HCT']))]['juris'].to_list()
# # Exclude any PAA (potential annexation area from this list); we will consider that unincorporated King County
# for i in king_city_list:
#     if 'PAA' in i:
#         king_city_list.remove(i)

# # Manually remove some anomalies:
# king_city_list.remove('North Highline')

# missing_city_list = []

king_city_list = ['Burien']

for city in king_city_list:
    if not os.path.isfile(os.path.join(output_dir,city+'.csv')):
        print(city)
        try:
            _gdf_shp = gdf_shp[gdf_shp['juris'] == city]
            df = evaluate_emissions(_gdf_shp, model_year)
            df.to_csv(os.path.join(output_dir,city+'.csv'), index=False)
        except:
            print('ERROR for: ' +city)
            missing_city_list.append(city)
            continue
    

In [35]:
df

,pollutantID,pollutant_name,veh_type,intrazonal_vmt,interzonal_vmt,start_tons,intrazonal_tons,interzonal_tons,total_daily_tons
0,1,Total Gaseous HCs,heavy,0.253858,2.648417e+06,0.023907,2.204663e-07,1.052126,1.076033
1,1,Total Gaseous HCs,light,703.023609,8.350151e+07,14.422171,9.803602e-05,8.246156,22.668425
2,1,Total Gaseous HCs,medium,1.580532,3.400392e+06,3.078701,7.051675e-07,0.735465,3.814167
3,1,Total Gaseous HCs,transit,0.000000,1.819112e+05,0.041907,0.000000e+00,0.346332,0.388240
4,2,CO,heavy,0.253858,2.648417e+06,0.085215,1.526871e-06,8.049700,8.134917
...,...,...,...,...,...,...,...,...,...
3,PM10,PM10 Total,transit,0.000000,5.457335e+05,0.000927,0.000000e+00,0.073888,0.074816
0,PM25,PM25 Total,heavy,0.761574,7.945250e+06,0.000204,1.586394e-07,0.803659,0.803863
1,PM25,PM25 Total,light,2109.070826,2.505045e+08,0.340221,1.443404e-05,0.948325,1.288560
2,PM25,PM25 Total,medium,4.741596,1.020117e+07,0.059894,1.252197e-07,0.129999,0.189893


In [121]:
# Sometimes there are database issues when loading data;
# If the initial pass did not work for all cities, try a second pass
# Usually this will catch any city that did not work on the first attempt

for city in missing_city_list:
# for city in ['Bothell','Medina','Milton','Newcastle']:
    # _gdf_shp = g?.path.join(output_dir,city+'.csv'), index=False)
    # print(city)
    try:
        _gdf_shp = gdf_shp[gdf_shp['juris'] == city]
        df = evaluate_emissions(_gdf_shp, model_year)
        df.to_csv(os.path.join(output_dir,city+'.csv'), index=False)
    except:
        print('ERROR for: ' +city)
        continue

2050
2050
2050
2050


### Calculate Emissions and VMT for Entirity of King County
The remainder between the county and city totals will be unincorporated areas

In [132]:
# The shapefile used for this exercise covers the entirity of King County
# any location in the inverse of the previously calculated cities will be considered unincorporated King County
_gdf_shp = gdf_shp[(gdf_shp['cnty_name'] == 'King')]
df = evaluate_emissions(_gdf_shp, model_year)


2018


In [133]:
df.to_csv(os.path.join(output_dir,'King County Total.csv'), index=False)

In [140]:
df = pd.read_csv(os.path.join(output_dir,'King County Total.csv'))
df['vmt'] = df['interzonal_vmt'] + df['intrazonal_vmt']
df = df.groupby(['pollutant_name','veh_type']).sum()[['total_daily_tons','vmt']].reset_index()
df.to_csv(os.path.join(output_dir,model_year+'_county_summary.csv'), index=False)

### Create Summary

In [124]:
# Summarize total emissions by each city
# Note that VMT will be listed for the same city and vehicle type combination multiple times (for each pollutant)

output_dir = r'C:\Workspace\aq_tool\output\\' + model_year

df = pd.DataFrame()

for city in king_city_list:
    _df = pd.read_csv(os.path.join(output_dir,city+'.csv'))
    _df['vmt'] = _df['interzonal_vmt'] + _df['intrazonal_vmt']
    _df = _df.groupby(['pollutant_name','veh_type']).sum()[['total_daily_tons','vmt']].reset_index()
    _df['city'] = city
    df = df.append(_df)
df.to_csv(os.path.join(output_dir,model_year+'_summary.csv'))

